In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/alice-in-wonderland/alice_in_wonderland.txt


# 3
Попробуйте изменить параметры нейронной сети, генерирующей текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший текст из получившихся и опишите предпринятые для его получения действия. Можно использовать текст другого произведения

In [2]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers import SimpleRNN, LSTM, GRU, Bidirectional
from keras.models import Sequential

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
import os 

In [10]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/alice-in-wonderland/alice_in_wonderland.txt


In [3]:
# построчное чтение из примера с текстом
with open("/kaggle/input/alice-in-wonderland/alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)
nb_chars

55

In [6]:
# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

In [7]:
# для удобства выберете фиксированную длину последовательность 10 символов
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=bool)
y = np.zeros((len(input_chars), nb_chars), dtype=bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [8]:
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 15 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

### Модель по умолчанию

In [9]:
%%time


model = Sequential()
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


# выполнение серий тренировочных и демонстрационных итераций
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]


    # для числа шагов предсказаний использование текущей тренируемой модели
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()


Итерация #: 0
1241/1241 [==============================] - 13s 6ms/step - loss: 2.3518
Генерация из посева: ry word yo
ry word you dout the has she has she has she has she has she has she has she has she has she has she has she h==================================================
Итерация #: 1
1241/1241 [==============================] - 7s 6ms/step - loss: 1.9769
Генерация из посева: erg.org 1.
erg.org 1..  jurt and the was the was the was the was the was the was the was the was the was the was the was ==================================================
Итерация #: 2
1241/1241 [==============================] - 7s 6ms/step - loss: 1.8088
Генерация из посева: rises and 
rises and the mack the mack the mack the mack the mack the mack the mack the mack the mack the mack the mack t==================================================
Итерация #: 3
1241/1241 [==============================] - 7s 6ms/step - loss: 1.6944
Генерация из посева: umped up a
umped up and the rabbit and the marter and t

### Увеличим количество эпох до 3 и итерации так же увеличим размер BATCH_SIZE что бы быстрее обучалось.. Заменим GRU на LSTM 

In [ ]:
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 256, 128
NUM_ITERATIONS = 25 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 3
NUM_PREDS_PER_EPOCH = 100

In [10]:
%%time
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 256, 128
NUM_ITERATIONS = 25 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 3
NUM_PREDS_PER_EPOCH = 100
# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''


model = Sequential()
model.add(
    LSTM(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="Adam")


# выполнение серий тренировочных и демонстрационных итераций
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]


    # для числа шагов предсказаний использование текущей тренируемой модели
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()

Итерация #: 0
Epoch 1/3
621/621 [==============================] - 5s 5ms/step - loss: 2.6224
Epoch 2/3
621/621 [==============================] - 3s 5ms/step - loss: 2.1674
Epoch 3/3
621/621 [==============================] - 3s 5ms/step - loss: 2.0203
Генерация из посева:  (b) alter
 (b) alter and the said the said the said the said the said the said the said the said the said the said the s==================================================
Итерация #: 1
Epoch 1/3
621/621 [==============================] - 3s 5ms/step - loss: 1.9157
Epoch 2/3
621/621 [==============================] - 3s 5ms/step - loss: 1.8281
Epoch 3/3
621/621 [==============================] - 3s 5ms/step - loss: 1.7551
Генерация из посева: some crumb
some crumber a the rome the more the more the more the more the more the more the more the more the more the m==================================================
Итерация #: 2
Epoch 1/3
621/621 [==============================] - 3s 5ms/step - loss: 1.6925
Epoch 2/3
62

## Увеличим NUM_PREDS_PER_EPOCH и HIDDEN_SIZE

In [13]:
%%time
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 256, 521
NUM_ITERATIONS = 25 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 3
NUM_PREDS_PER_EPOCH = 160
# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''


model = Sequential()
model.add(
    LSTM(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="Adam")


# выполнение серий тренировочных и демонстрационных итераций
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]


    # для числа шагов предсказаний использование текущей тренируемой модели
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()

Итерация #: 0
Epoch 1/3
621/621 [==============================] - 7s 8ms/step - loss: 2.4152
Epoch 2/3
621/621 [==============================] - 5s 8ms/step - loss: 1.9247
Epoch 3/3
621/621 [==============================] - 5s 8ms/step - loss: 1.6986
Генерация из посева:  thought, 
 thought, and she was to the said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock t==================================================
Итерация #: 1
Epoch 1/3
621/621 [==============================] - 5s 8ms/step - loss: 1.5357
Epoch 2/3
621/621 [==============================] - 5s 8ms/step - loss: 1.4027
Epoch 3/3
621/621 [==============================] - 5s 8ms/step - loss: 1.2916
Генерация из посева: ered about
ered about it was things and the caterpillar. what i should think in the little good as it was things and the caterpillar. what i should think in the little good as it wa=====================================

### Можель показала хорошие результаты и выводит вполне осознаный ответ

## Увеличим количество эпох до 7

In [14]:
%%time
# установка ряда метапамертров  для нейронной сети и процесса тренировки

BATCH_SIZE, HIDDEN_SIZE = 256, 521
NUM_ITERATIONS = 25 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 7
NUM_PREDS_PER_EPOCH = 160

# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''


model = Sequential()
model.add(
    LSTM(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="Adam")


# выполнение серий тренировочных и демонстрационных итераций
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]


    # для числа шагов предсказаний использование текущей тренируемой модели
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()

Итерация #: 0
Epoch 1/7
621/621 [==============================] - 7s 8ms/step - loss: 2.4262
Epoch 2/7
621/621 [==============================] - 5s 8ms/step - loss: 1.9316
Epoch 3/7
621/621 [==============================] - 5s 8ms/step - loss: 1.7056
Epoch 4/7
621/621 [==============================] - 5s 8ms/step - loss: 1.5420
Epoch 5/7
621/621 [==============================] - 5s 8ms/step - loss: 1.4104
Epoch 6/7
621/621 [==============================] - 5s 8ms/step - loss: 1.3007
Epoch 7/7
621/621 [==============================] - 5s 8ms/step - loss: 1.2001
Генерация из посева: ain! let m
ain! let me see if the same things about it, and she was not a minute or two the the white rabbit was the white rabbit was the white rabbit was the white rabbit was the w==================================================
Итерация #: 1
Epoch 1/7
621/621 [==============================] - 5s 8ms/step - loss: 1.1017
Epoch 2/7
621/621 [==============================] - 5s 8ms/step - loss: 1.0057

### Посмотрим как в тех же условиях покажет себя ванильная рекурентная сеть

In [19]:
%%time
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 256, 521
NUM_ITERATIONS = 25 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 5
NUM_PREDS_PER_EPOCH = 160
# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''


model = Sequential()
model.add(
    SimpleRNN(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="Adam")


# выполнение серий тренировочных и демонстрационных итераций
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]


    # для числа шагов предсказаний использование текущей тренируемой модели
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()

Итерация #: 0
Epoch 1/5
621/621 [==============================] - 4s 4ms/step - loss: 2.3540
Epoch 2/5
621/621 [==============================] - 2s 4ms/step - loss: 2.1134
Epoch 3/5
621/621 [==============================] - 3s 4ms/step - loss: 2.0643
Epoch 4/5
621/621 [==============================] - 2s 4ms/step - loss: 2.0071
Epoch 5/5
621/621 [==============================] - 2s 4ms/step - loss: 1.9476
Генерация из посева: ng? said t
ng? said the grapling to the grapling to the grapling to the grapling to the grapling to the grapling to the grapling to the grapling to the grapling to the grapling to t==================================================
Итерация #: 1
Epoch 1/5
621/621 [==============================] - 2s 4ms/step - loss: 1.8884
Epoch 2/5
621/621 [==============================] - 2s 4ms/step - loss: 1.8319
Epoch 3/5
621/621 [==============================] - 2s 4ms/step - loss: 1.7776
Epoch 4/5
621/621 [==============================] - 2s 4ms/step - loss: 1.7288

# Итог
Как видно увеличение итерации и эпох хорошо сказывается на генерации текста.
Из моделей лучше всех оказалась.  
Испльзование ванильных сетей показывает более худший результат чем GRU и LSTM. LSTM и GRU в срежнем имеют схожие результаты.